<a href="https://colab.research.google.com/github/sagepranjal/Car_price_predictor/blob/main/Car_price_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imported Libraries

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics

# Import the datset

In [ ]:
from google.colab import drive

drive.mount('/content/drive')


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Datasets/car_dekho.csv')

In [ ]:
df.head()


In [ ]:
df.shape
df.columns

Checking the unique values

In [ ]:
print(df['seller_type'].unique())
print(df['fuel'].unique())
print(df['transmission'].unique())
print(df['owner'].unique())


Creating the final dataset

In [ ]:
final_dataset = df[[ 'year', 'selling_price', 'km_driven', 'fuel', 'seller_type',
       'transmission', 'owner']]

In [ ]:
final_dataset.head()

In [ ]:
final_dataset['Current_year']=2020
final_dataset['no_years']=final_dataset['Current_year']-final_dataset['year']
final_dataset.head()


In [ ]:
final_dataset.drop(['Current_year'],axis=1,inplace=True)
final_dataset.drop(['year'],axis=1,inplace=True)
final_dataset.head()

In [ ]:
final_dataset=pd.get_dummies(final_dataset,drop_first=True)
final_dataset.head()

correlation matrix

In [ ]:
final_dataset.corr()

plotting the heat map

In [ ]:
mat = final_dataset.corr()
top_features = mat.index
plt.figure(figsize=(15,15))
g= sns.heatmap(final_dataset[top_features].corr(),annot=True,cmap="RdYlGn")

# Independent and dependent features

In [ ]:
X= final_dataset.iloc[:,1:]
Y=final_dataset.iloc[:,0] 

# Feature  importance

In [ ]:
model = ExtraTreesRegressor()
model.fit(X,Y)

print(model.feature_importances_)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [ ]:
#Randomized Search CV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [ ]:

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()

In [ ]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:

rf_random.fit(X_train,y_train)

In [ ]:
rf_random.best_params_

In [ ]:
predictions=rf_random.predict(X_test)
sns.distplot(y_test-predictions)

In [ ]:

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
plt.scatter(y_test,predictions)